In [1]:
import dask.dataframe as dd
import pandas as pd
import json
import os

/var/folders/xd/6cyv76d907x7b327h4r17nt40000gn/T/ipykernel_45250/4046248414.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use a new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

To disable this warning in the future, set dask config:

    # via Python
    >>> dask.config.set({'dataframe.query-planning-warning': False})

    # via

# Download and extract GTFS data from IDFM

In [2]:
# GTFS data (used for timetable)
STATIC_GTFS_URL="https://eu.ftp.opendatasoft.com/stif/GTFS/IDFM-gtfs.zip"
STATIC_GTFS_FILE_PATH="raw_data/gtfs.zip"
STATIC_GTFS_PATH="raw_data/gtfs"

!mkdir -p $STATIC_GTFS_PATH
!wget $STATIC_GTFS_URL -O $STATIC_GTFS_FILE_PATH
!unzip -o $STATIC_GTFS_FILE_PATH -d $STATIC_GTFS_PATH

--2024-03-26 19:09:14--  https://eu.ftp.opendatasoft.com/stif/GTFS/IDFM-gtfs.zip
Resolving eu.ftp.opendatasoft.com (eu.ftp.opendatasoft.com)... 52.18.186.238
Connecting to eu.ftp.opendatasoft.com (eu.ftp.opendatasoft.com)|52.18.186.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85610429 (82M) [application/zip]
Saving to: ‘raw_data/gtfs.zip’

raw_data/gtfs.zip   100%[===================>]  81,64M  33,9MB/s    in 2,4s    

2024-03-26 19:09:17 (33,9 MB/s) - ‘raw_data/gtfs.zip’ saved [85610429/85610429]

Archive:  raw_data/gtfs.zip
  inflating: raw_data/gtfs/transfers.txt  
  inflating: raw_data/gtfs/agency.txt  
  inflating: raw_data/gtfs/calendar_dates.txt  
  inflating: raw_data/gtfs/calendar.txt  
  inflating: raw_data/gtfs/stops.txt  
  inflating: raw_data/gtfs/trips.txt  
  inflating: raw_data/gtfs/routes.txt  
  inflating: raw_data/gtfs/stop_extensions.txt  
  inflating: raw_data/gtfs/stop_times.txt  
  inflating: raw_data/gtfs/pathways.txt  


# Loading and parsing GTFS data

In [3]:
calendar_path = os.path.join(STATIC_GTFS_PATH, 'calendar.txt')
calendar = pd.read_csv(calendar_path, dtype={'service_id': 'string',
                                             'monday': 'bool',
                                             'tuesday': 'bool',
                                             'wednesday': 'bool',
                                             'thursday': 'bool',
                                             'friday': 'bool',
                                             'saturday': 'bool',
                                             'sunday': 'bool',
                                             'start_date': 'int',
                                             'end_date': 'int'
                                             })
calendar = calendar.set_index('service_id')
calendar.head()

,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
service_id,,,,,,,,,
IDFM:1,False,True,True,True,True,False,False,20240326,20240405
IDFM:10,False,False,True,False,False,False,False,20240327,20240403
IDFM:890,True,True,True,False,True,True,False,20240326,20240424
IDFM:955,False,False,False,True,False,False,True,20240328,20240421
IDFM:1000,True,True,True,True,True,False,False,20240326,20240424


In [4]:
stops_path = os.path.join(STATIC_GTFS_PATH, 'stops.txt')
stops = pd.read_csv(stops_path, dtype={'stop_id': 'string',
                                       'stop_code': 'string',
                                       'stop_name': 'string',
                                       'stop_desc': 'string',
                                       'stop_lon': 'float',
                                       'stop_lat': 'float',
                                       'zone_id': 'string',
                                       'stop_url': 'string',
                                       'location_type': 'string',
                                       'parent_station': 'string',
                                       'stop_timezone': 'string',
                                       'level_id': 'string',
                                       'wheelchair_boarding': 'int',
                                       'platform_code': 'string', })
stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lon,stop_lat,zone_id,stop_url,location_type,parent_station,stop_timezone,level_id,wheelchair_boarding,platform_code
0,IDFM:monomodalStopPlace:46689,<NA>,Viroflay Rive Droite,<NA>,2.167655,48.805473,3,<NA>,0,IDFM:422420,<NA>,<NA>,1,<NA>
1,IDFM:monomodalStopPlace:46647,<NA>,Achères Ville,<NA>,2.077727,48.970644,5,<NA>,0,IDFM:73604,<NA>,<NA>,1,<NA>
2,IDFM:monomodalStopPlace:474069,<NA>,Lozère,<NA>,2.212718,48.705912,4,<NA>,0,IDFM:63029,<NA>,<NA>,1,<NA>
3,IDFM:monomodalStopPlace:474082,<NA>,Noisy-le-Grand - Mont d'Est,<NA>,2.550027,48.840885,4,<NA>,0,IDFM:412697,<NA>,<NA>,1,<NA>
4,IDFM:monomodalStopPlace:45102,<NA>,Châtelet les Halles,<NA>,2.346977,48.861745,1,<NA>,0,IDFM:474151,<NA>,<NA>,1,<NA>


In [5]:
trips_path = os.path.join(STATIC_GTFS_PATH, 'trips.txt')
trips = dd.read_csv(trips_path, dtype={'route_id': 'string',
                                       'service_id': 'string',
                                       'trip_id': 'string',
                                       'trip_headsign': 'string',
                                       'trip_short_name': 'string',
                                       'direction_id': 'string',
                                       'wheelchair_accessible': 'int',
                                       'bikes_allowed': 'int'})

trips['route_short_id'] = trips['route_id'].apply(lambda x: x.split(':')[-1],
                                                  meta=('route_id', 'string'))
trips.head()                                                  

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,route_short_id
0,IDFM:C00594,IDFM:1,IDFM:TRANSDEV_COTEAUX_DE_LA_MARNE:133170-C0059...,Plaine Haute,<NA>,0,NaN,NaN,1,0,C00594
1,IDFM:C00103,IDFM:1,IDFM:TRANSDEV_COEUR_ESSONNE:129405-C00103-1790...,Maison d'Arrêt des Femmes,<NA>,1,NaN,NaN,2,0,C00103
2,IDFM:C00031,IDFM:1,IDFM:N4_MOBILITES:102364-C00031-16410006,Gare d'Ozoir,<NA>,1,NaN,NaN,2,0,C00031
3,IDFM:C00031,IDFM:1,IDFM:N4_MOBILITES:102364-C00031-16410005,Gare d'Ozoir,<NA>,1,NaN,NaN,2,0,C00031
4,IDFM:C00031,IDFM:1,IDFM:N4_MOBILITES:102364-C00031-16410002,Gare d'Ozoir,<NA>,1,NaN,NaN,2,0,C00031


In [6]:
stop_times_path = os.path.join(STATIC_GTFS_PATH, 'stop_times.txt')
stop_times = dd.read_csv(stop_times_path, dtype={'trip_id': 'string',
                                                 'arrival_time': 'string',
                                                 'departure_time': 'string',
                                                 'stop_id': 'string',
                                                 'stop_sequence': 'int',
                                                 'pickup_type': 'int',
                                                 'drop_off_type': 'int',
                                                 'local_zone_id': 'string',
                                                 'stop_headsign': 'string',
                                                 'timepoint': 'string'})
stop_times.head()                                                 

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint
0,IDFM:TRANSDEV_COTEAUX_DE_LA_MARNE:133170-C0059...,07:18:00,07:18:00,IDFM:17398,0,0,1,<NA>,<NA>,1
1,IDFM:TRANSDEV_COTEAUX_DE_LA_MARNE:133170-C0059...,07:19:00,07:19:00,IDFM:17030,1,0,0,<NA>,<NA>,1
2,IDFM:TRANSDEV_COTEAUX_DE_LA_MARNE:133170-C0059...,07:20:00,07:20:00,IDFM:17027,2,0,0,<NA>,<NA>,1
3,IDFM:TRANSDEV_COTEAUX_DE_LA_MARNE:133170-C0059...,07:22:00,07:22:00,IDFM:21780,3,0,0,<NA>,<NA>,1
4,IDFM:TRANSDEV_COTEAUX_DE_LA_MARNE:133170-C0059...,07:25:00,07:25:00,IDFM:17042,4,0,0,<NA>,<NA>,1


# Get time table for relevant lines (metros, tramways, trains)
## Determine relevant lines

In [7]:
# Get all lines from GTFS
all_lines = set(trips['route_short_id'].values.compute())

# Get list of lines with shortest_paths computed
computed_lines_path = os.path.join('data', 'shortest_paths')
computed_lines = {x.split('.')[0] for x in os.listdir(computed_lines_path)}

# Get relevant lines
lines = computed_lines.intersection(all_lines)
print(lines)

/Users/bubu/Dev/IDFM-live-map/.venv/lib/python3.12/site-packages/dask/dataframe/core.py:3929: UserWarning: Dask currently has limited support for converting pandas extension dtypes to arrays. Converting string to object dtype.
  warnings.warn(


{'C01729', 'C01373', 'C01741', 'C01737', 'C01380', 'C01730', 'C01728', 'C01999', 'C01391', 'C01378', 'C01387', 'C01742', 'C01679', 'C01390', 'C01379', 'C02344', 'C01383', 'C02528', 'C01843', 'C01684', 'C01743', 'C01377', 'C01740', 'C01739', 'C01386', 'C01794', 'C01371', 'C01375', 'C01736', 'C01731', 'C01795', 'C01374', 'C01384', 'C01382', 'C01727', 'C01381', 'C01376', 'C02317', 'C01774', 'C01372', 'C02529', 'C01738', 'C01389'}


## Enrich trips with calendar data
Store dataframe in memory for faster join after.

In [8]:
trips = trips[trips['route_short_id'].isin(lines)]
trips = trips.set_index('service_id')
trips = trips.join(calendar, how='inner')
trips = trips.reset_index().set_index('trip_id')
trips.head()

,service_id,route_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,route_short_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
trip_id,,,,,,,,,,,,,,,,,,,
IDFM:KT9:132296-C02317-100-783,IDFM:103111,IDFM:C02317,Porte de Choisy,<NA>,0,NaN,NaN,1,0,C02317,True,True,True,True,True,True,False,20240326,20240424
IDFM:KT9:132296-C02317-1003-800,IDFM:10044,IDFM:C02317,Orly - Gaston Viens,<NA>,1,NaN,NaN,1,0,C02317,False,False,False,False,False,True,False,20240330,20240420
IDFM:KT9:132296-C02317-1012-800,IDFM:10044,IDFM:C02317,Porte de Choisy,<NA>,0,NaN,NaN,1,0,C02317,False,False,False,False,False,True,False,20240330,20240420
IDFM:KT9:132296-C02317-1023-800,IDFM:10044,IDFM:C02317,Porte de Choisy,<NA>,0,NaN,NaN,1,0,C02317,False,False,False,False,False,True,False,20240330,20240420
IDFM:KT9:132296-C02317-1026-800,IDFM:10044,IDFM:C02317,Orly - Gaston Viens,<NA>,1,NaN,NaN,1,0,C02317,False,False,False,False,False,True,False,20240330,20240420


In [9]:
# Store dataframe in memory
trips = trips.compute()

In [10]:
trips.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 100376 entries, IDFM:KT9:132296-C02317-100-783 to IDFM:TN:SNCF:fffe4326-9ab5-47f3-a4db-f81165736838
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   service_id             100376 non-null  string 
 1   route_id               100376 non-null  string 
 2   trip_headsign          100376 non-null  string 
 3   trip_short_name        45296 non-null   string 
 4   direction_id           100376 non-null  string 
 5   block_id               0 non-null       float64
 6   shape_id               0 non-null       float64
 7   wheelchair_accessible  100376 non-null  int64  
 8   bikes_allowed          100376 non-null  int64  
 9   route_short_id         100376 non-null  string 
 10  monday                 100376 non-null  bool   
 11  tuesday                100376 non-null  bool   
 12  wednesday              100376 non-null  bool   
 13  thursday              

## Enrich time table with trip data
Join stop_times dataframe with trips dataframe.

In [11]:
trips_id = set(trips.index.values)
stop_times = stop_times[stop_times['trip_id'].isin(trips_id)]
stop_times = stop_times.compute()

In [12]:
stop_times = stop_times.set_index('trip_id').join(trips, how='inner',
                                                  lsuffix='stop_times_',
                                                  rsuffix='trips_')
stop_times = stop_times.reset_index()
stop_times.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039904 entries, 0 to 2039903
Data columns (total 29 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trip_id                string 
 1   arrival_time           string 
 2   departure_time         string 
 3   stop_id                string 
 4   stop_sequence          int64  
 5   pickup_type            int64  
 6   drop_off_type          int64  
 7   local_zone_id          string 
 8   stop_headsign          string 
 9   timepoint              string 
 10  service_id             string 
 11  route_id               string 
 12  trip_headsign          string 
 13  trip_short_name        string 
 14  direction_id           string 
 15  block_id               float64
 16  shape_id               float64
 17  wheelchair_accessible  int64  
 18  bikes_allowed          int64  
 19  route_short_id         string 
 20  monday                 bool   
 21  tuesday                bool   
 22  wednesday         

## Saving data for each line
Parquet is faster to load/write than csv.

In [13]:
for line in lines:
    print(f'Saving line {line}')
    l_stop_times = stop_times[stop_times['route_short_id'] == line]
    save_directory = os.path.join('data', 'timetable')
    if not os.path.exists(save_directory):
        os.mkdir(save_directory)
    l_stop_times.to_parquet(os.path.join(save_directory, line))

Saving line C01729
Saving line C01373
Saving line C01741
Saving line C01737
Saving line C01380
Saving line C01730
Saving line C01728
Saving line C01999
Saving line C01391
Saving line C01378
Saving line C01387
Saving line C01742
Saving line C01679
Saving line C01390
Saving line C01379
Saving line C02344
Saving line C01383
Saving line C02528
Saving line C01843
Saving line C01684
Saving line C01743
Saving line C01377
Saving line C01740
Saving line C01739
Saving line C01386
Saving line C01794
Saving line C01371
Saving line C01375
Saving line C01736
Saving line C01731
Saving line C01795
Saving line C01374
Saving line C01384
Saving line C01382
Saving line C01727
Saving line C01381
Saving line C01376
Saving line C02317
Saving line C01774
Saving line C01372
Saving line C02529
Saving line C01738
Saving line C01389
